# Asset Straddle Tickers Module

This notebook demonstrates the asset straddle ticker utilities from `specparser.amt.asset_straddle_tickers`.

## Table of Contents

1. [Purpose](#Purpose) - What this module does
2. [Setup](#Setup) - Imports and configuration
3. [Inputs](#Inputs) - Function parameters
4. [Getting Tickers for a Single Asset](#Getting-Tickers-for-a-Single-Asset) - `get_asset_straddle_tickers`
5. [Understanding Ticker Types](#Understanding-Ticker-Types) - Vol and Hedge configuration
6. [Futures Ticker Construction](#Futures-Ticker-Construction) - `make_fut_ticker`
7. [Split Tickers](#Split-Tickers) - Handling ticker transitions
8. [Finding Tickers Across Date Ranges](#Finding-Tickers-Across-Date-Ranges) - `find_assets_straddles_tickers`
9. [Analyzing Ticker Patterns](#Analyzing-Ticker-Patterns) - Examples and analysis
10. [Benchmark](#Benchmark) - Performance benchmarks
11. [Cache Management](#Cache-Management) - `clear_ticker_caches`
12. [Command Line Interface](#Command-Line-Interface) - CLI usage
13. [Summary](#Summary) - Function reference

## Purpose

When pricing straddles, you need specific tickers for:
- **Volatility (vol)**: Implied volatility data for option pricing
- **Hedge**: The underlying hedge instrument (futures, spot, CDS, etc.)

This module extracts these tickers based on asset configuration and straddle parameters.

## Setup

In [1]:
import pandas as pd
from specparser.amt.asset_straddle_tickers import (
    get_asset_straddle_tickers,
    find_assets_straddles_tickers,
    split_ticker,
    make_fut_ticker,
    clear_ticker_caches,
)
from specparser.amt import (
    get_asset,
    find_assets,
    show_table,
)

# Data path (relative to this notebook)
AMT_PATH = "../data/amt.yml"

## Inputs

The main function `get_asset_straddle_tickers(asset, strym, ntrc, amt_path)` takes:

| Parameter | Description | Example |
|-----------|-------------|--------|
| `asset` | Asset underlying | `"CL Comdty"`, `"SPX Index"` |
| `strym` | Year-month string | `"2024-06"` |
| `ntrc` | Entry code | `"N"` (near) or `"F"` (far) |
| `amt_path` | Path to AMT YAML file | `"data/amt.yml"` |

### Entry Code (ntrc)

| Code | Meaning | Vol Field Used |
|------|---------|----------------|
| `N` | Near month entry | `Vol.Near` |
| `F` | Far month entry | `Vol.Far` |

---
## Getting Tickers for a Single Asset

`get_asset_straddle_tickers(asset, strym, ntrc, amt_path)` returns a table with columns:
- `name`: Ticker role (`vol`, `hedge`, `hedge1`, etc.)
- `ticker`: The ticker symbol
- `field`: The data field to fetch

In [2]:
# CL Comdty (Crude Oil) - June 2024, Near month entry
tickers = get_asset_straddle_tickers("CL Comdty", "2024-06", "N", AMT_PATH)
print("CL Comdty - June 2024, Near entry:")
show_table(tickers)

CL Comdty - June 2024, Near entry:


,name,ticker,field
0,vol,CL1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
1,hedge,CLN2024 Comdty,PX_LAST


In [3]:
# Same asset with Far month entry
tickers = get_asset_straddle_tickers("CL Comdty", "2024-06", "F", AMT_PATH)
print("CL Comdty - June 2024, Far entry:")
show_table(tickers)

CL Comdty - June 2024, Far entry:


,name,ticker,field
0,vol,CL1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
1,hedge,CLN2024 Comdty,PX_LAST


In [4]:
# SPX Index (Equity)
tickers = get_asset_straddle_tickers("SPX Index", "2024-06", "N", AMT_PATH)
print("SPX Index - June 2024:")
show_table(tickers)

SPX Index - June 2024:


,name,ticker,field
0,vol,SPX Index,30DAY_IMPVOL_100.0%MNY_DF
1,hedge,SPX Index,PX_LAST


In [5]:
# EURUSD Curncy
tickers = get_asset_straddle_tickers("EURUSD Curncy", "2024-06", "N", AMT_PATH)
print("EURUSD Curncy - June 2024:")
show_table(tickers)

EURUSD Curncy - June 2024:


,name,ticker,field
0,vol,EURUSDV1M CMPN Curncy,PX_LAST
1,hedge,EURUSD Curncy,PX_LAST


In [6]:
# Rate asset (may have CDS hedges)
tickers = get_asset_straddle_tickers("IBOXUMAE Curncy", "2024-06", "N", AMT_PATH)
print("IBOXUMAE Curncy - June 2024:")
show_table(tickers)

IBOXUMAE Curncy - June 2024:


,name,ticker,field
0,vol,CREDIT.CVOL.CDX_NA_IG.1M.CVOL_ATM.CVOL_VOL.CVO...,none
1,hedge,IBOXUMAE CBBT Curncy,PX_LAST
2,hedge1,USSWAP5 CMPN Curncy,PX_LAST


---
## Understanding Ticker Types

The tickers come from the asset's Vol and Hedge configuration in the AMT file.

In [7]:
# Vol and Hedge configuration for CL Comdty
asset_data = get_asset(AMT_PATH, "CL Comdty")

print("CL Comdty Vol Configuration:")
for k, v in asset_data["Vol"].items():
    print(f"  {k}: {v}")

print("\nCL Comdty Hedge Configuration:")
for k, v in asset_data["Hedge"].items():
    print(f"  {k}: {v}")

CL Comdty Vol Configuration:
  Source: BBG
  Ticker: CL1 Comdty
  Near: 1ST_MTH_IMPVOL_100.0%MNY_DF
  Far: 2ND_MTH_IMPVOL_100.0%MNY_DF

CL Comdty Hedge Configuration:
  Source: fut
  generic: CL1 Comdty
  fut_code: CL
  fut_month_map: GHJKMNQUVXZF
  min_year_offset: 0
  market_code: Comdty


In [8]:
# Compare with SPX Index (non-futures hedge)
asset_data = get_asset(AMT_PATH, "SPX Index")

print("SPX Index Vol Configuration:")
for k, v in asset_data["Vol"].items():
    print(f"  {k}: {v}")

print("\nSPX Index Hedge Configuration:")
for k, v in asset_data["Hedge"].items():
    print(f"  {k}: {v}")

SPX Index Vol Configuration:
  Source: BBG
  Ticker: SPX Index
  Near: 30DAY_IMPVOL_100.0%MNY_DF
  Far: 60DAY_IMPVOL_100.0%MNY_DF

SPX Index Hedge Configuration:
  Source: nonfut
  Ticker: SPX Index
  Field: PX_LAST


---
## Futures Ticker Construction

For assets with futures hedges, tickers are constructed from components using `make_fut_ticker()`.

In [9]:
# Construct a futures ticker manually
ticker = make_fut_ticker(
    fut_code="CL",
    fut_month_map="GHJKMNQUVXZF",
    min_year_offset=0,
    market_code="Comdty",
    qualifier="",
    year=2024,
    month=6
)
print(f"CL futures for June 2024: {ticker}")

CL futures for June 2024: CLN2024 Comdty


In [10]:
# Generate tickers for a full year
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", 
          "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

print("CL Comdty futures calendar for 2024:")
print("-" * 40)
for month_num, month_name in enumerate(months, 1):
    ticker = make_fut_ticker(
        fut_code="CL",
        fut_month_map="GHJKMNQUVXZF",
        min_year_offset=0,
        market_code="Comdty",
        qualifier="",
        year=2024,
        month=month_num
    )
    print(f"{month_name:3s}: {ticker}")

CL Comdty futures calendar for 2024:
----------------------------------------
Jan: CLG2024 Comdty
Feb: CLH2024 Comdty
Mar: CLJ2024 Comdty
Apr: CLK2024 Comdty
May: CLM2024 Comdty
Jun: CLN2024 Comdty
Jul: CLQ2024 Comdty
Aug: CLU2024 Comdty
Sep: CLV2024 Comdty
Oct: CLX2024 Comdty
Nov: CLZ2024 Comdty
Dec: CLF2025 Comdty


---
## Split Tickers

Some tickers change at a specific date (e.g., LIBOR to SOFR transition).

Format: `ticker1:YYYY-MM:ticker2`

In [11]:
# LIBOR to SOFR transition at 2023-06
split_ticker_str = "USSWAP5 CMPN Curncy:2023-06:USOSFR5 Curncy"

print(f"Split ticker: {split_ticker_str}")
print()
print("Resolution at different dates:")
for year, month in [(2023, 1), (2023, 5), (2023, 6), (2023, 7), (2024, 1)]:
    result = split_ticker(split_ticker_str, year, month)
    print(f"  {year}-{month:02d}: {result}")

Split ticker: USSWAP5 CMPN Curncy:2023-06:USOSFR5 Curncy

Resolution at different dates:
  2023-01: USOSFR5 Curncy
  2023-05: USOSFR5 Curncy
  2023-06: USOSFR5 Curncy
  2023-07: USSWAP5 CMPN Curncy
  2024-01: USSWAP5 CMPN Curncy


---
## Finding Tickers Across Date Ranges

`find_assets_straddles_tickers(pattern, ntry, xpry, amt_path)` finds all unique tickers needed for assets matching a pattern over a date range.

In [12]:
# All unique tickers for CL Comdty in H1 2024
tickers = find_assets_straddles_tickers("^CL Comdty$", "2024-01", "2024-06", AMT_PATH)
print(f"CL Comdty H1 2024: {len(tickers['rows'])} unique tickers")
show_table(tickers)

CL Comdty H1 2024: 8 unique tickers


,name,ticker,field
0,vol,CL1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
1,hedge,CLG2024 Comdty,PX_LAST
2,vol,CL1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,hedge,CLH2024 Comdty,PX_LAST
4,hedge,CLJ2024 Comdty,PX_LAST
5,hedge,CLK2024 Comdty,PX_LAST
6,hedge,CLM2024 Comdty,PX_LAST
7,hedge,CLN2024 Comdty,PX_LAST


In [13]:
# All commodities for full year 2024
tickers = find_assets_straddles_tickers("Comdty$", "2024-01", "2024-12", AMT_PATH)
print(f"All commodities 2024: {len(tickers['rows'])} unique tickers")
show_table(tickers)

All commodities 2024: 176 unique tickers


,name,ticker,field
0,vol,LA1 Comdty,1ST_MTH_IMPVOL_100.0%MNY_DF
1,hedge,LAF2024 Comdty,PX_LAST
2,vol,LA1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
3,hedge,LAG2024 Comdty,PX_LAST
4,hedge,LAH2024 Comdty,PX_LAST
...,...,...,...
171,vol,OE1 Comdty,2ND_MTH_IMPVOL_100.0%MNY_DF
172,hedge,OEM2024 Comdty,PX_LAST
173,hedge,OEU2024 Comdty,PX_LAST
174,hedge,OEZ2024 Comdty,PX_LAST


In [14]:
# Equities for H1 2024
tickers = find_assets_straddles_tickers("(Index|Equity)$", "2024-01", "2024-06", AMT_PATH)
print(f"Equities H1 2024: {len(tickers['rows'])} unique tickers")
show_table(tickers)

Equities H1 2024: 174 unique tickers


,name,ticker,field
0,vol,HYG US Equity,1ST_MTH_IMPVOL_100.0%MNY_DF
1,hedge,HYG US Equity,PX_LAST
2,vol,LQD US Equity,1ST_MTH_IMPVOL_100.0%MNY_DF
3,hedge,LQD US Equity,PX_LAST
4,vol,EMB US Equity,1ST_MTH_IMPVOL_100.0%MNY_DF
...,...,...,...
169,hedge,CSCO US Equity,PX_LAST
170,vol,DVN US Equity,30DAY_IMPVOL_100.0%MNY_DF
171,hedge,DVN US Equity,PX_LAST
172,vol,LOW US Equity,30DAY_IMPVOL_100.0%MNY_DF


---
## Analyzing Ticker Patterns

In [15]:
# Analyze ticker roles
from collections import Counter

tickers = find_assets_straddles_tickers(".", "2024-01", "2024-06", AMT_PATH)

roles = Counter(row[0] for row in tickers["rows"])
print("Tickers by role:")
for role, count in roles.most_common():
    print(f"  {role}: {count}")

Tickers by role:
  hedge: 225
  vol: 215
  hedge1: 18
  hedge2: 16
  hedge3: 16
  hedge4: 16


In [16]:
# Compare ticker counts across asset classes
patterns = [
    ("Comdty$", "Commodities"),
    ("Index$", "Indices"),
    ("Curncy$", "Currencies"),
    ("Equity$", "Equities"),
]

print("Unique ticker counts for 2024 (full year):")
print("-" * 50)
for pattern, name in patterns:
    tickers = find_assets_straddles_tickers(pattern, "2024-01", "2024-12", AMT_PATH)
    n_assets = len(find_assets(AMT_PATH, pattern, live_only=True)["rows"])
    print(f"{name:15s}: {len(tickers['rows']):4d} tickers from {n_assets:3d} assets")

Unique ticker counts for 2024 (full year):
--------------------------------------------------
Commodities    :  176 tickers from  18 assets
Indices        :   20 tickers from  10 assets
Currencies     :  177 tickers from  75 assets
Equities       :  154 tickers from  77 assets


In [17]:
# Track hedge ticker changes over time for CL Comdty
print("CL Comdty hedge tickers by month (2024):")
print("-" * 50)

for month in range(1, 13):
    ym = f"2024-{month:02d}"
    tickers = get_asset_straddle_tickers("CL Comdty", ym, "N", AMT_PATH)
    hedge = next((row[1] for row in tickers["rows"] if row[0] == "hedge"), "N/A")
    print(f"  {ym}: {hedge}")

CL Comdty hedge tickers by month (2024):
--------------------------------------------------
  2024-01: CLG2024 Comdty
  2024-02: CLH2024 Comdty
  2024-03: CLJ2024 Comdty
  2024-04: CLK2024 Comdty
  2024-05: CLM2024 Comdty
  2024-06: CLN2024 Comdty
  2024-07: CLQ2024 Comdty
  2024-08: CLU2024 Comdty
  2024-09: CLV2024 Comdty
  2024-10: CLX2024 Comdty
  2024-11: CLZ2024 Comdty
  2024-12: CLF2025 Comdty


---
## Cache Management

---
## Benchmark

**Note:** `find_assets_straddles_tickers` returns only tickers needed for **straddle pricing**:
- **Vol** ticker (Near or Far based on entry code)
- **Hedge** tickers (hedge, hedge1, hedge2, etc.)

This excludes **Market** tickers (asset prices) which are included in `find_tickers` (from `tickers.ipynb`). The two functions serve different purposes and return different counts.

In [18]:
# Benchmark: Get all unique tickers for straddle pricing (2001-2025)
import timeit

# Parameters - use YYYY-MM format for start/end
START_YM, END_YM = "2001-01", "2025-12"
PATTERN = "."
N_RUNS = 3

# Warmup
clear_ticker_caches()
_ = find_assets_straddles_tickers(PATTERN, START_YM, END_YM, AMT_PATH)

# Get result stats
result = find_assets_straddles_tickers(PATTERN, START_YM, END_YM, AMT_PATH)
n_tickers = len(result['rows'])

# Count by role
from collections import Counter
name_idx = result['columns'].index('name')
roles = Counter(row[name_idx] for row in result['rows'])

print(f"All live assets ({START_YM} to {END_YM}): {n_tickers} unique straddle tickers")
print(f"  vol: {roles['vol']}, hedge: {roles['hedge']}, hedge1-4: {roles['hedge1'] + roles['hedge2'] + roles['hedge3'] + roles['hedge4']}")
print()

# Benchmark (with cache - typical use case)
times_cached = timeit.repeat(
    lambda: find_assets_straddles_tickers(PATTERN, START_YM, END_YM, AMT_PATH),
    repeat=N_RUNS,
    number=1
)

# Benchmark (without cache - cold start)
def cold_run():
    clear_ticker_caches()
    return find_assets_straddles_tickers(PATTERN, START_YM, END_YM, AMT_PATH)

times_cold = timeit.repeat(cold_run, repeat=N_RUNS, number=1)

print(f"Benchmark ({N_RUNS} runs):")
print("-" * 50)
print(f"{'With cache (warm):':<25} {min(times_cached)*1000:6.1f}ms")
print(f"{'Without cache (cold):':<25} {min(times_cold)*1000:6.1f}ms")

All live assets (2001-01 to 2025-12): 4552 unique straddle tickers
  vol: 1097, hedge: 3388, hedge1-4: 67

Benchmark (3 runs):
--------------------------------------------------
With cache (warm):        1520.1ms
Without cache (cold):     1528.8ms


In [19]:
# Clear caches (useful after AMT file changes)
clear_ticker_caches()
print("Ticker caches cleared")

Ticker caches cleared


---
## Command Line Interface

### Get tickers for a single asset

```bash
uv run python -m specparser.amt.asset_straddle_tickers \
    data/amt.yml --get "CL Comdty" --ym 2024-06 --ntrc N
```

```
name	ticker	field
vol	CL1 Comdty	1ST_MTH_IMPVOL_100.0%MNY_DF
hedge	CLN2024 Comdty	PX_LAST
```

```bash
# Far month entry
uv run python -m specparser.amt.asset_straddle_tickers \
    data/amt.yml --get "CL Comdty" --ym 2024-06 --ntrc F
```

```
name	ticker	field
vol	CL1 Comdty	2ND_MTH_IMPVOL_100.0%MNY_DF
hedge	CLN2024 Comdty	PX_LAST
```

### Find unique tickers across date ranges

```bash
uv run python -m specparser.amt.asset_straddle_tickers \
    data/amt.yml --find "^CL Comdty$" --start 2024-01 --end 2024-06
```

```
name	ticker	field
vol	CL1 Comdty	1ST_MTH_IMPVOL_100.0%MNY_DF
hedge	CLG2024 Comdty	PX_LAST
vol	CL1 Comdty	2ND_MTH_IMPVOL_100.0%MNY_DF
hedge	CLH2024 Comdty	PX_LAST
hedge	CLJ2024 Comdty	PX_LAST
hedge	CLK2024 Comdty	PX_LAST
hedge	CLM2024 Comdty	PX_LAST
hedge	CLN2024 Comdty	PX_LAST

Total unique tickers: 8
```

### Help

```bash
uv run python -m specparser.amt.asset_straddle_tickers --help
```

```
usage: asset_straddle_tickers.py [-h] [--get ASSET] [--ym YYYY-MM]
                                 [--ntrc {N,F}] [--find PATTERN]
                                 [--start YYYY-MM] [--end YYYY-MM]
                                 amt_path

Asset straddle ticker utilities

positional arguments:
  amt_path         Path to AMT YAML file

options:
  -h, --help       show this help message and exit
  --get ASSET      Get tickers for a single asset
  --ym YYYY-MM     Year-month for --get (e.g., 2024-06)
  --ntrc {N,F}     Entry code: N=near, F=far (default: N)
  --find PATTERN   Find tickers for assets matching regex pattern
  --start YYYY-MM  Start year-month for --find
  --end YYYY-MM    End year-month for --find
```

---
## Summary

### Main Functions

| Function | Description |
|----------|-------------|
| `get_asset_straddle_tickers(asset, strym, ntrc, amt_path)` | Get tickers for one asset/straddle |
| `find_assets_straddles_tickers(pattern, ntry, xpry, amt_path)` | Find unique tickers for assets over date range |

### Helper Functions

| Function | Description |
|----------|-------------|
| `split_ticker(ticker, year, month)` | Resolve split ticker format |
| `make_fut_ticker(...)` | Construct futures ticker from components |
| `clear_ticker_caches()` | Clear memoization cache |

### Output Columns

| Column | Description |
|--------|-------------|
| `name` | Ticker role: `vol`, `hedge`, `hedge1`, `hedge2`, `hedge3`, `hedge4` |
| `ticker` | The ticker symbol to query |
| `field` | The data field to fetch (empty for calculated tickers) |